# 🔍 Building an AI-Powered Web Search Agent with OpenAI and Tavily 🚀

Hey there! Welcome to this exciting guide where we'll create something awesome - a smart search agent that combines the power of OpenAI's language models with Tavily's search capabilities! 🌟 

## 🎯 What We'll Build

We're going to create a super cool search agent that can:
1. 🌐 Search the web in real-time for accurate information
2. 🧠 Use OpenAI's powerful GPT models to understand and process search results
3. ⚡ Provide contextual and up-to-date responses to queries

## ✅ Prerequisites

Before we jump in, make sure you have these things ready:
- 🔑 An OpenAI API key
- 🎯 A Tavily API key (get one at tavily.com)

## 🎮 Part 1: Setting Up Our Environment

First things first - let's get our tools ready! We'll need to install the Tavily Python package to interact with their search API:


In [1]:
# Install necessary libraries
!pip install streamlit openai requests python-dotenv

In [2]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


In [3]:
if TAVILY_API_KEY:
    print("✅ Tavily API Key loaded successfully.")
else:
    print("❌ Error: Tavily API Key not found. Check your .env file.")


✅ Tavily API Key loaded successfully.



## 🛠️ Part 2: Building Our Search Tools

Let's create the foundation of our search agent! We'll define a set of tools that our AI can use to search the web:

In [4]:
import json
from openai import OpenAI
from tavily import TavilyClient
import pprint

In [5]:
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# Search query Ombu Urban Lab
import requests

def web_search(city, topic, timeframe, doc_type, num_results=5):
    # Construct a smarter query including the selected document type
    query = f"{doc_type} about {topic} in {city} during {timeframe}"

    url = "https://api.tavily.com/search"
    headers = {"Authorization": f"Bearer {TAVILY_API_KEY}"}
    payload = {"query": query, "num_results": num_results}

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json().get("results", [])
    else:
        print("Error:", response.text)
        return []



In [6]:
web_search("Tokyo", "Green Areas", "2020-2025", num_results=5)

[{'title': 'PDF',
  'url': 'https://phpmyadmin.cloud1.glc.org/Download_PDFS/virtual-library/U000002/2_Tokyo_S_Urban_Growth_Urban_Form_And_Sustainability.pdf',
  'content': "Tokyo, urban growth, urban form, sustainability, megacity, density, transportation, energy, waste management, environmental impact, urban planning, green spaces, resilience, smart city, innovation. Tokyo, one of the world's largest and most densely populated cities, has experienced phenomenal growth over the past century. This growth has",
  'score': 0.32649738,
  'raw_content': None},
 {'title': "Policy Assessment of Japan's 'Decarbonisation-Leading Regions'",
  'url': 'https://ietresearch.onlinelibrary.wiley.com/doi/10.1049/smc2.70002?af=R',
  'content': "Policy Assessment of Japan's ‘Decarbonisation‐Leading Regions’ - Guo - 2025 - IET Smart Cities - Wiley Online Library First, we look at the policy stances of the decarbonising regions of Japan and other regions, aiming to provide new perspectives for formulating 

In [7]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for urban planning information, reports, studies, or academic papers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query including topic, city, and timeframe"}
                },
                "required": ["query"]
            },
        },
    },
]



## 🎓 Part 3: Creating Our AI Agent

Now comes the exciting part! Let's create our AI agent that can understand questions and use our search tools to find answers:


In [8]:
from datetime import datetime

messages = [
    {
        "role": "system",
        "content": f"""
        You are a helpful Urban Research Assistant specializing in mobility, urban planning, green spaces, and public policies.
        
        Your mission is to:
        1. Search the web for serious sources like urban reports, policy documents, and academic research papers.
        2. List the best findings together with their links and bullet points showing their key insights.
            
        Make as many tool calls as needed to find good-quality sources before responding.
        Focus on reliable information, avoid blogs, tourism websites, and commercial content.
        
        Current date: {datetime.now().strftime('%Y-%m-%d')}
        """
    },
    {
        "role": "user",
        "content": "Find urban planning trends related to shared mobility in Utrecht for 2020-2025."
    }
]


In [9]:
def invoke_model(messages):
    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [10]:
# Initialize the OpenAI client
client = OpenAI()

# Make a ChatGPT API call with tool calling
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    tools=TOOLS,
    messages=messages
)

response = completion.choices[0].message
pprint.pprint(response.tool_calls)

# Parse the response to get the tool call arguments
if response.tool_calls:
    # Process each tool call
    for tool_call in response.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "search_web":
            print("Searching for", tool_call_arguments)
            search_results = search_web(tool_call_arguments["query"])
            messages.append({"role": "assistant", "content": f"{tool_call_arguments["query"]}: {search_results}"})
    print(invoke_model(messages))

else:
    # If there are no tool calls, return the response content
    print(response.content)

[ChatCompletionMessageToolCall(id='call_izi9Wv31oRPDJ26RtEe1obPD', function=Function(arguments='{"query":"shared mobility trends urban planning Utrecht 2020-2025"}', name='web_search'), type='function')]
Here are several reliable sources that discuss urban planning trends related to shared mobility in Utrecht from 2020 to 2025, along with key insights from each source.

### 1. **Utrecht City Mobility Plan 2020-2025**
- **Source**: Utrecht City Government Document
- **Link**: [Utrecht Mobility Plan](https://www.utrecht.nl)
- **Key Insights**:
  - Emphasis on transitioning to sustainable mobility modes, including shared mobility.
  - Introduction of incentives for electric shared vehicles to reduce emissions.
  - Expansion of shared bike and e-scooter networks throughout the city.
  - Integration of shared mobility solutions with public transport options to enhance accessibility.

### 2. **Shared Mobility in Dutch Urban Areas: An Overview**
- **Source**: The Netherlands Institute for Tra

In [11]:
# create a function call the OpenAi API 

def agent(messages):

    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        tools=TOOLS, # here we pass the tools to the LLM
        messages=messages
    )

    # Get the response from the LLM
    response = completion.choices[0].message

    # Parse the response to get the tool call arguments
    if response.tool_calls:
        # Process each tool call
        for tool_call in response.tool_calls:
            # Get the tool call arguments
            tool_call_arguments = json.loads(tool_call.function.arguments)
            if tool_call.function.name == "save_memory":
                return save_memory(tool_call_arguments["memory"])
            elif tool_call.function.name == "web_search":
                search_results = search_web(tool_call_arguments["query"])
                messages.append({"role": "assistant", "content": f"Here are the search results: {search_results}"})
                return invoke_model(messages)
    else:
        # If there are no tool calls, return the response content
        return response.content

In [12]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for urban planning information, reports, studies, or academic papers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query including topic, city, and timeframe"}
                },
                "required": ["query"]
            },
        },
    },
]


In [13]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for urban planning information, reports, studies, or academic papers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query including topic, city, and timeframe"}
                },
                "required": ["query"]
            },
        },
    },
]


In [16]:
messages

[{'role': 'system',
  'content': '\n        You are a helpful Urban Research Assistant specializing in mobility, urban planning, green spaces, and public policies.\n        \n        Your mission is to:\n        1. Search the web for serious sources like urban reports, policy documents, and academic research papers.\n        2. List the best findings together with their links and bullet points showing their key insights.\n            \n        Make as many tool calls as needed to find good-quality sources before responding.\n        Focus on reliable information, avoid blogs, tourism websites, and commercial content.\n        \n        Current date: 2025-04-28\n        '},
 {'role': 'user',
  'content': 'Find urban planning trends related to shared mobility in Utrecht for 2020-2025.'}]